<a href="https://colab.research.google.com/github/AsraniSanjana/All_Codes/blob/main/All_Semester_Codes/ML_sem7/models/practice_id3_c4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
class Node:
    """Contains the information of the node and another nodes of the Decision Tree."""

    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None

class DecisionTreeClassifier:
    """Decision Tree Classifier using ID3 algorithm."""

    def __init__(self, X, feature_names, labels):
        self.X = X  # features or predictors
        self.feature_names = feature_names  # name of the features
        self.labels = labels  # categories
        self.labelCategories = list(set(labels))  # unique categories
        # number of instances of each category
        self.labelCategoriesCount = [list(labels).count(x) for x in self.labelCategories]
        self.node = None  # nodes
        # calculate the initial entropy of the system
        self.entropy = self._get_entropy([x for x in range(len(self.labels))])


    def _get_entropy(self, x_ids):
        """ Calculates the entropy.
        Parameters
        __________
        :param x_ids: list, List containing the instances ID's
        __________
        :return: entropy: float, Entropy.
        """
        # sorted labels by instance id
        labels = [self.labels[i] for i in x_ids]
        # count number of instances of each category
        label_count = [labels.count(x) for x in self.labelCategories]
        # calculate the entropy for each category and sum them
        entropy = sum([-count / len(x_ids) * math.log(count / len(x_ids), 2)
                      if count else 0
                      for count in label_count
                      ])

        return entropy

    def _get_information_gain(self, x_ids, feature_id):
        """Calculates the information gain for a given feature based on its entropy and the total entropy of the system.
        Parameters
        __________
        :param x_ids: list, List containing the instances ID's
        :param feature_id: int, feature ID
        __________
        :return: info_gain: float, the information gain for a given feature.
        """
        # calculate total entropy
        info_gain = self._get_entropy(x_ids)
        # store in a list all the values of the chosen feature
        x_features = [self.X[x][feature_id] for x in x_ids]
        # get unique values
        feature_vals = list(set(x_features))
        # get frequency of each value
        feature_v_count = [x_features.count(x) for x in feature_vals]
        # get the feature values ids
        feature_v_id = [
            [x_ids[i]
            for i, x in enumerate(x_features)
            if x == y]
            for y in feature_vals
        ]

        # compute the information gain with the chosen feature
        info_gain_feature = sum([v_counts / len(x_ids) * self._get_entropy(v_ids)
                            for v_counts, v_ids in zip(feature_v_count, feature_v_id)])

        info_gain = info_gain - info_gain_feature

        return info_gain


    def _get_feature_max_information_gain(self, x_ids, feature_ids):
        """Finds the attribute/feature that maximizes the information gain.
        Parameters
        __________
        :param x_ids: list, List containing the samples ID's
        :param feature_ids: list, List containing the feature ID's
        __________
        :returns: string and int, feature and feature id of the feature that maximizes the information gain
        """
        # get the entropy for each feature
        features_entropy = [self._get_information_gain(x_ids, feature_id) for feature_id in feature_ids]
        # find the feature that maximises the information gain
        max_id = feature_ids[features_entropy.index(max(features_entropy))]

        return self.feature_names[max_id], max_id


    def id3(self):
        """Initializes ID3 algorithm to build a Decision Tree Classifier.
        :return: None
        """
        # assign an unique number to each instance
        x_ids = [x for x in range(len(self.X))]
        # assign an unique number to each featuer
        feature_ids = [x for x in range(len(self.feature_names))]
        # define node variable - instance of the class Node
        self.node = self._id3_recv(x_ids, feature_ids, self.node)


    def _id3_recv(self, x_ids, feature_ids, node):
        """ID3 algorithm. It is called recursively until some criteria is met.
        Parameters
        __________
        :param x_ids: list, list containing the samples ID's
        :param feature_ids: list, List containing the feature ID's
        :param node: object, An instance of the class Nodes
        __________
        :returns: An instance of the class Node containing all the information of the nodes in the Decision Tree
        """
        if not node:
            node = Node()  # initialize nodes
        # sorted labels by instance id
        labels_in_features = [self.labels[x] for x in x_ids]
        # if all the example have the same class (pure node), return node
        if len(set(labels_in_features)) == 1:
            node.value = self.labels[x_ids[0]]
            return node
        # if there are not more feature to compute, return node with the most probable class
        if len(feature_ids) == 0:
            node.value = max(set(labels_in_features), key=labels_in_features.count)  # compute mode
            return node
        # else...
        # choose the feature that maximizes the information gain
        best_feature_name, best_feature_id = self._get_feature_max_information_gain(x_ids, feature_ids)
        node.value = best_feature_name
        node.childs = []
        # value of the chosen feature for each instance
        feature_values = list(set([self.X[x][best_feature_id] for x in x_ids]))
        # loop through all the values
        for value in feature_values:
            child = Node()
            child.value = value  # add a branch from the node to each feature value in our feature
            node.childs.append(child)  # append new child node to current node
            child_x_ids = [x for x in x_ids if self.X[x][best_feature_id] == value]
            if not child_x_ids:
                child.next = max(set(labels_in_features), key=labels_in_features.count)
                print('')
            else:
                if feature_ids and best_feature_id in feature_ids:
                    to_remove = feature_ids.index(best_feature_id)
                    feature_ids.pop(to_remove)
                # recursively call the algorithm
                child.next = self._id3_recv(child_x_ids, feature_ids, child.next)

    # if all the example have the same class (pure node), return node
                if len(set(labels_in_features)) == 1:
                    node.value = self.labels[x_ids[0]]
                    return node

                # if there are not more feature to compute, return node with the most probable class
                if len(feature_ids) == 0:
                    node.value = max(set(labels_in_features), key=labels_in_features.count)  # compute mode
                    return node

                  # else...
        # choose the feature that maximizes the information gain
                best_feature_name, best_feature_id = self._get_feature_max_information_gain(x_ids, feature_ids)
                node.value = best_feature_name
                node.childs = []
                # value of the chosen feature for each instance
                feature_values = list(set([self.X[x][best_feature_id] for x in x_ids]))
                # loop through all the values
                for value in feature_values:
                    child = Node()
                    child.value = value  # add a branch from the node to each feature value in our feature
                    node.childs.append(child)  # append new child node to current node
                    child_x_ids = [x for x in x_ids if self.X[x][best_feature_id] == value] # instances that take the branch
                    if not child_x_ids:
                        child.next = max(set(labels_in_features), key=labels_in_features.count)
                        print('')
                    else:
                        if feature_ids and best_feature_id in feature_ids:
                            to_remove = feature_ids.index(best_feature_id)
                            feature_ids.pop(to_remove)
                        # recursively call the algorithm
                        child.next = self._id3_recv(child_x_ids, feature_ids, child.next)
                return node
        return node

In [130]:
import pandas as pd
import math

# Define the Node class and DecisionTreeClassifier class here

# Load the dataset
data_df = pd.read_csv("Buy_Computer.csv")


# Extract features, feature_names, and labels from the dataset
X = data_df[['age', 'income', 'student', 'credit_rating']].values
feature_names = ['age', 'income', 'student', 'credit_rating']
labels = data_df['Buy_Computer'].values

# Create an instance of DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(X, feature_names, labels)

# Run the ID3 algorithm
dt_classifier.id3()


In [132]:
import pandas as pd
import math

data_df = pd.read_csv("Buy_Computer.csv") # Buy_computer here is the dataset name

class_labels = data_df['Buy_Computer'] # Buy_Computer here is the last colmn name : class label : yes or no
dt_classifier = DecisionTreeClassifier(None,None,class_labels)  # Pass None for X and feature_names


**FOR ID3:**


1.   ENTROPY
2.   INFO GAIN
3.   GAIN

**FOR C4.5:**

1.   GAIN RATIO
2.   SPLIT INFO


**FOR CART:**

1.   GINI INDEX
2.   DELTA GINI

In [142]:
def getGain(attribute_name):
    attribute_column = data_df[attribute_name]
    class_labels = data_df['Buy_Computer']

    # Create an instance of DecisionTreeClassifier
    dt_classifier = DecisionTreeClassifier(None, None, class_labels)  # Pass None for X and feature_names

    # Calculate entropy for the attribute values
    attribute_values = attribute_column.unique()

    entropies = []
    weighted_entropies = []
    values_counts = {}
    for value in attribute_values:
        subset_indices = attribute_column[attribute_column == value].index
        entropy = dt_classifier._get_entropy(subset_indices)
        entropies.append(entropy)
        weight = len(subset_indices) / len(attribute_column)
        weighted_entropy = entropy * weight
        weighted_entropies.append(weighted_entropy)
        values_counts[value] = len(subset_indices)
        print(f"Entropy for {attribute_name} = {value}: {entropy:.4f} , weight = {weight:.4f}")


    # Calculate information gain for the attribute
    total_entropy = dt_classifier._get_entropy(attribute_column.index)
    total_weighted_entropy = sum(weighted_entropies)
    info_gain = total_weighted_entropy

    # Calculate and display the final gain for the attribute
    dataset_entropy = dt_classifier._get_entropy(attribute_column.index)
    final_gain = dataset_entropy - info_gain


    # Calculate split info for the attribute
    total_tuples = len(data_df)
    split_info = 0
    for value_count in values_counts.values():
        ratio = value_count / total_tuples
        split_info -= ratio * math.log2(ratio)

    gain_ratio = final_gain / split_info

    # Print the Gain Ratio
    print(f"Gain Ratio({attribute_name}) = {final_gain:.4f} / {split_info:.4f} = {gain_ratio:.4f}")



    # Print the values counts and entropies in a table format
    print("\nValues Counts and Entropies:")
    print(f"{attribute_name}\tBuys_Comp = Yes\tBuys_Comp = No\tTotal\t\tEntropy")
    for value in values_counts:
        yes_count = sum((data_df['Buy_Computer'][index] == 'yes') for index in data_df[data_df[attribute_name] == value].index)
        no_count = values_counts[value] - yes_count
        total_count = values_counts[value]
        entropy = entropies[attribute_values.tolist().index(value)]
        print(f"{value}\t\t\t{yes_count}\t\t\t{no_count}\t\t{total_count}\t\t{entropy:.4f}")



    print(f"\nHence, Gain({attribute_name}) becomes {dataset_entropy:.4f} - {info_gain:.4f} = {final_gain:.4f}")
    print(f"Split Info({attribute_name}) = {split_info:.4f}")



# Call the function for 'income' attribute
getGain('age')
getGain('income')
getGain('student')
getGain('credit_rating')


Entropy for age = youth: 0.9710 , weight = 0.3571
Entropy for age = middle_age: 0.0000 , weight = 0.2857
Entropy for age = senior: 0.9710 , weight = 0.3571
Gain Ratio(age) = 0.2467 / 1.5774 = 0.1564

Values Counts and Entropies:
age	Buys_Comp = Yes	Buys_Comp = No	Total		Entropy
youth			2			3		5		0.9710
middle_age			4			0		4		0.0000
senior			3			2		5		0.9710

Hence, Gain(age) becomes 0.9403 - 0.6935 = 0.2467
Split Info(age) = 1.5774
Entropy for income = high: 1.0000 , weight = 0.2857
Entropy for income = medium: 0.9183 , weight = 0.4286
Entropy for income = low: 0.8113 , weight = 0.2857
Gain Ratio(income) = 0.0292 / 1.5567 = 0.0188

Values Counts and Entropies:
income	Buys_Comp = Yes	Buys_Comp = No	Total		Entropy
high			2			2		4		1.0000
medium			4			2		6		0.9183
low			3			1		4		0.8113

Hence, Gain(income) becomes 0.9403 - 0.9111 = 0.0292
Split Info(income) = 1.5567
Entropy for student = no: 0.9852 , weight = 0.5000
Entropy for student = yes: 0.5917 , weight = 0.5000
Gain Ratio(student) 